# Regression Model

### Importing Libraries

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
os.chdir("drive/MyDrive/DL_Project")

In [ ]:
from Dataset import MyDataset
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Implementing Model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        resnet = models.resnet18(pretrained=True)  # Load pre-trained ResNet18
        # Remove the fully connected layers of ResNet18
        self.resnet_features = nn.Sequential(*list(resnet.children())[:-2])

        # Additional layers for adaptation to match the previous output size
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512,64)
        self.fc2 = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.resnet_features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor

        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = CNN()
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CNN(
  (resnet_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

### Setting Hyperparameters

In [ ]:
num_epochs=10
batch_size=8
learning_rate=0.00001

### Defining Loss Function and Optimizer

In [ ]:
criterion=nn.MSELoss()
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)

### Training & Validating Model

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_dataset=MyDataset("Data/train_set.csv",transform=transform)
train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
valid_dataset=MyDataset("Data/validation_set.csv",transform=transform)
valid_dataloader=torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for i, (images, target) in enumerate(train_dataloader):
        images = images.reshape(-1,3,4000,3000).to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        avg_loss=total_loss/len(train_dataloader)

    # Performing validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0

        for i, (val_images, val_target) in enumerate(valid_dataloader):
            val_images = val_images.reshape(-1,3,4000,3000).to(device)
            val_target = val_target.to(device)

            val_output = model(val_images)
            val_loss += criterion(val_output, val_target).item()

            average_val_loss = val_loss / len(valid_dataloader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.4f}, Validation Loss: {average_val_loss:.4f}')

OutOfMemoryError: ignored

### Saving Model Weights

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))

### Testing Model

In [ ]:
test_dataset=MyDataset("Data/test_set.csv",transform=transform)
test_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
correct=[]
predicted=[]
model.eval()
with torch.no_grad():
    test_loss = 0.0

    for i, (test_images, test_target) in enumerate(test_dataloader):
        test_images = test_images.reshape(-1,3,4000,3000).to(device)
        test_target = test_target.to(device)

        test_output = model(test_images)
        test_loss += criterion(test_output, test_target).item()

        correct.append(test_target)
        predicted.append(test_output)

        average_test_loss = test_loss / len(test_dataloader)

    print(f'Average Loss: {average_test_loss}')

### Sample Output

In [ ]:
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

img = Image.open('Data/FruitBox/Image_482.jpg')

plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
img = transform(Image.open(os.path.join("Data","FruitBox/Image_482.jpg")).convert("RGB"))

test_images = img.reshape(-1,3,4000,3000).to(device)
test_target = target.to(device)
test_output = model(test_images)

print(f'Predicted output : {test_output.item()} , Actual : {1.555}')